# Notebook - Análise da evasão

Dados do Instituto de Ciências Exatas (IE).

Importando bibliotecas:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.metrics
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

import pre_process
import utils

%matplotlib inline

 ### Leitura dos dados

In [2]:
import pre_process

DATA_FILE = '../data/ie_data.csv'
data_0 = pd.read_csv(DATA_FILE, sep=';', low_memory=False)

In [7]:
data_1.columns

Index(['public_school', 'approved_credits', 'dropout', 'course', 'female',
       'quota', 'programming_subject', 'distance'],
      dtype='object')

In [8]:
data_1.head()

,public_school,approved_credits,dropout,course,female,quota,programming_subject,distance
302,False,16.0,False,ciência da computação,False,False,0,5182
444,True,10.0,True,ciência da computação,True,True,0,22956
509,False,6.0,False,ciência da computação,False,False,0,8492
654,False,14.0,False,ciência da computação,False,False,0,17782
968,False,20.0,False,ciência da computação,False,False,0,4556


## Pré-Processamento

In [3]:
# Pre-Process
columns = []
data_0 = pre_process.format_data(data_0)
data_0 = pre_process.public_school(data_0, columns)
data_0 = pre_process.credits(data_0, columns)
data_0 = pre_process.dropout(data_0, columns)
data_0 = pre_process.course(data_0, columns)
data_0 = pre_process.gender(data_0, columns)
data_0 = pre_process.quota(data_0, columns)
# data_0 = pre_process.ira(data_0, columns)
data_0 = pre_process.programming_subjects(data_0, columns)
data_0 = pre_process.cep(data_0, columns)

In [4]:
data_1 = data_0.copy()[columns].drop_duplicates()

In [5]:
data_course = pre_process.divide_course(data_1)

## Testes

In [ ]:
data_0['course'].unique()

In [ ]:
columns.append(['codigo_disciplina', 'creditos_disciplina', 'mencao_disciplina'])
data_course = pre_process.divide_course(data_0)
course = 'engenharia mecatrônica'

## Processamento

In [ ]:
attr = 'dropout'
X = data1.drop(columns=[attr])
y = data1[attr]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
model = CatBoostClassifier()
# model.fit(X_train, y_train, cat_features=[2], plot=True)
model.fit(X_train, y_train, cat_features=[1])

In [ ]:
predictions = model.predict(X_test)
predictions = [x == 'True' for x in predictions]
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
print("Recall score:", sklearn.metrics.recall_score(y_test, predictions))
print("Precusion score:", sklearn.metrics.precision_score(y_test, predictions))

In [ ]:
def plot_feature_importance(importance,names,model_type):
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    plt.figure(figsize=(10,8))
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

plot_feature_importance(model.get_feature_importance(), X.columns,'CATBOOST')

## Using SHAP

In [ ]:
from multiprocessing import Pool

import shap
explainer = shap.TreeExplainer(model)
start_index = 0
end_index = 1000
shap_values = explainer.shap_values(X_test[start_index:end_index])

In [ ]:
shap.initjs()

In [ ]:
id = 4
shap.force_plot(explainer.expected_value,
                shap_values[id:id+1],
                X_test[id:id+1])

In [ ]:
shap.force_plot(explainer.expected_value,
                shap_values,
                X_test[start_index:end_index])

In [ ]:
shap.summary_plot(shap_values, X[start_index:end_index], plot_size=(20,20))

In [ ]:
shap.dependence_plot('programming_subject', shap_values, X_test[start_index:end_index], interaction_index='distance')

In [ ]:
shap.summary_plot(shap_values, X_test[start_index:end_index])

## Plot Student's House Coordinates

In [ ]:
attr = ['aluno', 'cep', 'dropout', 'course']
data_map = data_0.copy()[attr].drop_duplicates()
data_map_course = pre_process.divide_course(data_map)

utils.plot_coordinates(data_map, 'all courses')
for course, data_course in data_map_course.items():
    utils.plot_coordinates(data_course, course)

In [ ]:
sns.displot(data_map, x=x, y=y, hue='dropout', height=6)

In [ ]:
p = sns.jointplot(data=data_map,x=x, y=y,kind='kde')

In [ ]:
sns.kdeplot(x=x,y=y,hue=c,
            shade=True,
            thresh=False
)